# YouTube Downloads

In [212]:
from pytube import YouTube
path = "/Users/jacobhansen/Desktop/test/"
yt = YouTube('https://www.youtube.com/watch?v=Uz3LExBIdks')
yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution')[-1].download(output_path = path)

'/Users/jacobhansen/Desktop/test/Driving into Salt Lake City.mp4'

# Canny Filter on Image

In [ ]:
import cv2 as cv

In [81]:
# Read the original image
img = cv.imread('/Users/jacobhansen/Desktop/canyonPhoto.png')

# Display original image
cv.imshow('Original', img)

cv.waitKey(0)

# Convert to graycsale
img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

# Blur the image for better edge detection
img_blur = cv.GaussianBlur(img_gray, (3,3), 0)

# Canny Edge Detection
edges_clean = cv.Canny(image=img_gray, threshold1=100, threshold2=200) # Canny Edge Detection
cv.imshow('Canny Edge Detection - No Blur', edges_clean) # Display Canny Edge Detection Image

cv.waitKey(0)

edges = cv.Canny(image=img_blur, threshold1=100, threshold2=200) # Canny Edge Detection
cv.imshow('Canny Edge Detection - Medium Blur', edges) # Display Canny Edge Detection Image

cv.waitKey(0) # Wait Until Key is Given

img_blur_heavy = cv.GaussianBlur(img_gray, (5,5), 0)
edges = cv.Canny(image=img_blur_heavy, threshold1=100, threshold2=200) # Canny Edge Detection
cv.imshow('Canny Edge Detection - Heavy Blur', edges) # Display Canny Edge Detection Image

cv.waitKey(0) # Wait Until Key is Given

cv.destroyAllWindows()
cv.waitKey(1) # needed to close the windows


-1

# Canny Filter on Video

In [2]:
import cv2 as cv
import numpy as np

In [19]:
def canny(img, blur = 3, threshold1 = 100, threshold2 = 200):
    """
    Recieves a color image and return a c
    """
    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # Blur the image for better edge detection
    img_blur = cv.GaussianBlur(img_gray, (blur,blur), 0)

    # Canny Edge Detection
    edges_clean = cv.Canny(image=img_blur, threshold1=threshold1, threshold2=threshold2) # Canny Edge Detection

    # return to same format as input
    img_complete = cv.cvtColor(edges_clean,cv.COLOR_GRAY2RGB)
    return img_complete

In [36]:
def save_canny_video(path, save_name, blur = 3):
    """
    recieves a path to a video with a blur and a name for which to save it. 
    """
    assert blur%2 != 0, "Blur input must be an odd number"

    # set filter to desired output, filter called as filter(img, blur)
    filter = canny

    # initialize video capture
    cap = cv.VideoCapture(path)

    # Get the Default resolutions
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    frame_rate = cap.get(5)

    # Define the codec and filename.
    out = cv.VideoWriter(save_name+".avi",cv.VideoWriter_fourcc('M','J','P','G'), frame_rate, (frame_width, frame_height)) #

    while(cap.isOpened()):
        # read frames, ret is true/false if a frame was returned (ie havne't reached the end)
        ret, frame = cap.read()
        
        if ret==True:
            # run canny filter on frame
            lines = filter(frame, blur)

            # write the  frame
            out.write(lines)

            # display image while 
            # cv.imshow('frame',lines)
            # if cv.waitKey(1) & 0xFF == ord('q'):
            #     break
        else:
            break

    # Release everything if job is finished
    cap.release()
    # out.release()

    cv.destroyAllWindows() # close window 
    cv.waitKey(1) # needed for mac OS to close the window

In [37]:
path = "data examples/Driving into Salt Lake City - Long.mp4"
output_path = "data examples/Driving into SLC - blur 1 - Long"
save_canny_video(path, output_path, 1)

# Video Stabilization

In [41]:
from vidstab import VidStab

line_path = output_path+".avi"
# Using defaults
stabilizer = VidStab(kp_method='FAST', threshold=42, nonmaxSuppression=False)
stabilizer.stabilize(input_path=line_path, output_path='stable_video.avi')

# Homography Transform

In [42]:
######################################################
## DUMMY POINTS -- ENTER YOUR MEASUREMENTS HERE
PTS_IMAGE_PLANE = [[240, 372],
                    [2, 372],
                    [332, 192],
                    [258, 192],
                    [189, 192],
		    [403, 192],
		    [476, 192],
                    [332, 207],
                    [235, 207],
                    [142, 207],
                    [432, 207],
                    [532, 207],
                    [332, 238],
                    [173, 238],
                    [9, 238],
                    [499, 238],
                    [658, 238]] # dummy points
######################################################

# PTS_GROUND_PLANE units are in inches
# camera looks along positive x axis with positive y axis to left

######################################################
## DUMMY POINTS -- ENTER YOUR MEASUREMENTS HERE
PTS_GROUND_PLANE = [[0, 0],
		    [0, -13],
		    [-60, 0],
                    [-60, -15],
                    [-60, -30],
                    [-60, 15],
		    [-60, 30],
		    [-40, 0],
		    [-40, -15],
                    [-40, -30],
                    [-40, 15],
                    [-40, 30],
		    [-20, 0],
                    [-20, -15],
                    [-20, -30],
                    [-20, 15],
                    [-20, 30]]
 # dummy points
######################################################

METERS_PER_INCH = 0.0254


class HomographyTransformer:
    def __init__(self):

        if not len(PTS_GROUND_PLANE) == len(PTS_IMAGE_PLANE):
            print("ERROR: PTS_GROUND_PLANE and PTS_IMAGE_PLANE should be of same length")

        #Initialize data into a homography matrix

        np_pts_ground = np.array(PTS_GROUND_PLANE)
        np_pts_ground = np_pts_ground * METERS_PER_INCH
        np_pts_ground = np.float32(np_pts_ground[:, np.newaxis, :])

        np_pts_image = np.array(PTS_IMAGE_PLANE)
        np_pts_image = np_pts_image * 1.0
        np_pts_image = np.float32(np_pts_image[:, np.newaxis, :])

        self.h, err = cv.findHomography(np_pts_image, np_pts_ground)



    def transformUvToXy(self, u, v):
        """
        u and v are pixel coordinates.
        The top left pixel is the origin, u axis increases to right, and v axis
        increases down.
        Returns a normal non-np 1x2 matrix of xy displacement vector from the
        camera to the point on the ground plane.
        Camera points along positive x axis and y axis increases to the left of
        the camera.
        Units are in meters.
        """
        homogeneous_point = np.array([[u], [v], [1]])
        xy = np.dot(self.h, homogeneous_point)
        scaling_factor = 1.0 / xy[2, 0]
        homogeneous_xy = xy * scaling_factor
        x = homogeneous_xy[0, 0]
        y = homogeneous_xy[1, 0]
        return x, y

transformer = HomographyTransformer()